In [1]:
import numpy as np
import keras
from tensorflow.keras.layers import Dense, Input, LSTM
from tensorflow.keras.models import Model

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "ben-eng/ben.txt"

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 4332
Number of unique input tokens: 71
Number of unique output tokens: 91
Max sequence length for inputs: 96
Max sequence length for outputs: 95


In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [7]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [9]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [10]:
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [11]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
55/55 [==============================] - 50s 835ms/step - loss: 1.4343 - accuracy: 0.7490 - val_loss: 1.6977 - val_accuracy: 0.6723
Epoch 2/100
55/55 [==============================] - 45s 819ms/step - loss: 0.7786 - accuracy: 0.8192 - val_loss: 1.6898 - val_accuracy: 0.6747
Epoch 3/100
55/55 [==============================] - 45s 816ms/step - loss: 0.7030 - accuracy: 0.8255 - val_loss: 1.4357 - val_accuracy: 0.6813
Epoch 4/100
55/55 [==============================] - 44s 807ms/step - loss: 0.6327 - accuracy: 0.8408 - val_loss: 1.1872 - val_accuracy: 0.7009
Epoch 5/100
55/55 [==============================] - 46s 834ms/step - loss: 0.5626 - accuracy: 0.8555 - val_loss: 1.0744 - val_accuracy: 0.7235
Epoch 6/100
55/55 [==============================] - 45s 822ms/step - loss: 0.6172 - accuracy: 0.8552 - val_loss: 1.0157 - val_accuracy: 0.7326
Epoch 7/100
55/55 [==============================] - 44s 801ms/step - loss: 0.4944 - accuracy: 0.8690 - val_loss: 0.9715 - val_accuracy:

55/55 [==============================] - 41s 744ms/step - loss: 0.1461 - accuracy: 0.9588 - val_loss: 1.0680 - val_accuracy: 0.7803
Epoch 58/100
55/55 [==============================] - 40s 736ms/step - loss: 0.1418 - accuracy: 0.9598 - val_loss: 1.0805 - val_accuracy: 0.7808
Epoch 59/100
55/55 [==============================] - 40s 734ms/step - loss: 0.1395 - accuracy: 0.9605 - val_loss: 1.0798 - val_accuracy: 0.7765
Epoch 60/100
55/55 [==============================] - 40s 734ms/step - loss: 0.1373 - accuracy: 0.9614 - val_loss: 1.0719 - val_accuracy: 0.7811
Epoch 61/100
55/55 [==============================] - 40s 737ms/step - loss: 0.1336 - accuracy: 0.9621 - val_loss: 1.1083 - val_accuracy: 0.7799
Epoch 62/100
55/55 [==============================] - 40s 734ms/step - loss: 0.1312 - accuracy: 0.9626 - val_loss: 1.1083 - val_accuracy: 0.7783
Epoch 63/100
55/55 [==============================] - 41s 737ms/step - loss: 0.1289 - accuracy: 0.9633 - val_loss: 1.1281 - val_accuracy: 0.774

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


In [12]:
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Go.
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Go.
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Run!
Decoded sentence: এইগুলো টমে।

-
Input sentence: Run!
Decoded sentence: এইগুলো টমে।

-
Input sentence: Who?
Decoded sentence: এই বইগুলো টিকিনো।

-
Input sentence: Fire!
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Help!
Decoded sentence: আমাকে বলুন কি ঘটেছিলো।

-
Input sentence: Help!
Decoded sentence: আমাকে বলুন কি ঘটেছিলো।

-
Input sentence: Stop!
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Stop!
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Stop!
Decoded sentence: আমাকে একলা ছেড়ে যেও না।

-
Input sentence: Hello!
Decoded sentence: এই বইগুলো সহিও না।

-
Input sentence: I see.
Decoded sentence: আমি তোমাকে পেটাতে পারি।

-
Input sentence: I try.
Decoded sentence: আমি তোমাকে পেটাতে পারি।

-
Input sentence: Smile.
Decode